In [20]:
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import h5py
import numpy as np
import re

In [ ]:
img = np.array(Image.open("./content/datasets/data_in/objects/real/26_cam-image_array_.jpg"))
obj_ids = np.unique(img)
print(obj_ids)

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def map_values(value):
    if value in [128]:
        return 1
    elif value in [255]:
        return 2
    else:
        return 0
    return value

def map_to_r(number):
    colorr = [255, 128, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 255, 128, 255, 0, 128]
    if number<20:
      return int(colorr[number])
    else:
      return 0

def map_to_g(number):

    colorg = [0, 128, 255, 128, 0, 0, 128, 0, 0, 128, 0, 128, 0, 128, 255, 128, 0, 128, 255, 128]

    if number<20:
      return int(colorg[number])
    else:
      return 0

def map_to_b(number):

    colorb = [0, 0, 0, 128, 128, 255, 255, 128, 0, 128, 128, 255, 128, 0, 0, 0, 128, 255, 255, 128]

    if number<20:
      return int(colorb[number])
    else:
      return 255

def load_data(images_folder):
  raw_image=[]
  segmentation_mask=[]
  vectorized_map = np.vectorize(map_values)
  image_files = sorted(os.listdir(images_folder+"/"),key=natural_sort_key)
  for image_file in image_files:
      image = cv2.imread(images_folder+"/"+image_file)
      raw_image.append(image)
  mask_files = sorted(os.listdir(images_folder+"_mask_retouched/"),key=natural_sort_key)
  for mask_file in mask_files:
      if mask_file[0]!=".":
          image = cv2.imread(images_folder+"_mask_retouched/"+mask_file)
          semantic_id=vectorized_map(image[:,:,0])
          # colored_mask = np.zeros_like(raw_image[0])
          # colored_mask[:, :, 0] = np.vectorize(map_to_b)(semantic_id)
          # colored_mask[:, :, 1] = np.vectorize(map_to_g)(semantic_id)
          # colored_mask[:, :, 2] = np.vectorize(map_to_r)(semantic_id)
          segmentation_mask.append(semantic_id)
  
  return raw_image,segmentation_mask


raw_image_sim = []
raw_image_real = []


print("-STARTING real- ")
images_folder = f"./content/datasets/processed_data/real"
raw_image_real,segmentation_mask_real=load_data(images_folder)

print("-STARTING sim- ")
images_folder = f"./content/datasets/processed_data/sim_rectified"
raw_image_sim,segmentation_mask_sim=load_data(images_folder)

raw_images_real={}
raw_images_real["0001"]=raw_image_real
raw_images_sim={}
raw_images_sim["0001"]=raw_image_sim

segmentation_masks_real={}
segmentation_masks_real["0001"]=segmentation_mask_real
segmentation_masks_sim={}
segmentation_masks_sim["0001"]=segmentation_mask_sim



In [ ]:
for i in range(0, 100,10):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(2, 2, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs[0,0].imshow(cv2.cvtColor(raw_image_real[i], cv2.COLOR_BGR2RGB))
      axs[0,0].set_title('Real Image')
      axs[0,1].imshow(cv2.cvtColor(raw_image_sim[i], cv2.COLOR_BGR2RGB))
      axs[0,1].set_title('Sim Image')
      axs[1,0].imshow(segmentation_mask_real[i])
      axs[1,0].set_title('Real Mask')
      axs[1,1].imshow(segmentation_mask_sim[i])
      axs[1,1].set_title('Sim mask')
      obj_ids = np.unique(segmentation_mask_sim[i])
      print(obj_ids)
      for ax in axs.flat:
          ax.set_xticks([])
          ax.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [25]:
import pickle
with open('./content/datasets/pkl_out_GAN_train/raw_image_sim.pkl', 'wb') as file:
    pickle.dump(raw_images_sim, file)
with open('./content/datasets/pkl_out_GAN_train/raw_image_real.pkl', 'wb') as file:
    pickle.dump(raw_images_real, file)
with open('./content/datasets/pkl_out_GAN_train/mask_sim.pkl', 'wb') as file:
    pickle.dump(segmentation_masks_sim, file)
with open('./content/datasets/pkl_out_GAN_train/mask_real.pkl', 'wb') as file:
    pickle.dump(segmentation_masks_real, file)

In [26]:
import pickle
with open('./content/datasets/pkl_out_GAN_train/raw_image_sim.pkl', 'rb') as file:
    loaded_images_sim = pickle.load(file)

with open('./content/datasets/pkl_out_GAN_train/raw_image_real.pkl', 'rb') as file:
    loaded_images_real = pickle.load(file)

with open('./content/datasets/pkl_out_GAN_train/mask_sim.pkl', 'rb') as file:
    loaded_masks_sim = pickle.load(file)

with open('./content/datasets/pkl_out_GAN_train/mask_real.pkl', 'rb') as file:
    loaded_masks_real = pickle.load(file)

In [ ]:
for i in range(0, 100,10):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(2, 2, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs[0,0].imshow(cv2.cvtColor(loaded_images_real["0001"][i], cv2.COLOR_BGR2RGB))
      axs[0,0].set_title('Real Image')
      axs[0,1].imshow(cv2.cvtColor(loaded_images_sim["0001"][i], cv2.COLOR_BGR2RGB))
      axs[0,1].set_title('Sim Image')
      axs[1,0].imshow(loaded_masks_real["0001"][i])
      axs[1,0].set_title('Real Image')
      axs[1,1].imshow(loaded_masks_sim["0001"][i])
      axs[1,1].set_title('Sim Image')
      for ax in axs.flat:
          ax.set_xticks([])
          ax.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [29]:
def save_dictionary_to_h5(file_path, data_dict):
    with h5py.File(file_path, 'w') as hf:
        for key, array_list in data_dict.items():
            # Create a group for each key in the dictionary
            group = hf.create_group(key)
            for i, arr in enumerate(array_list):
                # Save each array as a dataset within the group
                group.create_dataset(f'array_{i}', data=arr)



file_path = './content/datasets/h5_out_GAN_train/raw_image_sim.h5'
save_dictionary_to_h5(file_path, loaded_images_sim)

file_path = './content/datasets/h5_out_GAN_train/raw_image_real.h5'
save_dictionary_to_h5(file_path, loaded_images_real)

file_path = './content/datasets/h5_out_GAN_train/semantic_id_list_sim.h5'
save_dictionary_to_h5(file_path, loaded_masks_sim)

file_path = './content/datasets/h5_out_GAN_train/semantic_id_list_real.h5'
save_dictionary_to_h5(file_path, loaded_masks_real)